In [1]:
import os
import keras
os.environ['KERAS_BACKEND']='tensorflow'

Using TensorFlow backend.
E:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

In [2]:
#importing libraries
import math
import keras
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense , BatchNormalization , Dropout , Activation
from keras.layers import LSTM , GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.optimizers import Adam , SGD , RMSprop

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv('BBCA.csv')

In [5]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-13,5000.0,5100.0,4975.0,5000.0,4287.987305,9781000.0
1,2010-01-14,4950.0,5150.0,4950.0,4950.0,4245.106934,5679000.0
2,2010-01-15,4950.0,5050.0,4950.0,4950.0,4245.106934,5158000.0
3,2010-01-18,5000.0,5100.0,4950.0,5050.0,4330.866211,2525500.0
4,2010-01-19,4975.0,5150.0,4975.0,4975.0,4266.546875,6175000.0


The profit or loss calculation is usually determined by the closing price of a stock for the day, hence we will consider the closing price as the target variable and drop the other columns.

In [6]:
data.drop(['Adj Close','Open','Low','High','Volume'],axis=1,inplace=True)

In [7]:
data.head()

,Date,Close
0,2010-01-13,5000.0
1,2010-01-14,4950.0
2,2010-01-15,4950.0
3,2010-01-18,5050.0
4,2010-01-19,4975.0


Now, the dtype of date is object. We'll convert that into date-time and will make it as the index for our dataframe.

In [8]:
data['Date'] = pd.to_datetime(data['Date'], infer_datetime_format=True)


In [9]:
data.index=data.Date

In [10]:
data.drop('Date', axis=1,inplace=True)

In [11]:
data.head()

,Close
Date,
2010-01-13,5000.0
2010-01-14,4950.0
2010-01-15,4950.0
2010-01-18,5050.0
2010-01-19,4975.0


In [12]:
dataset=data.values

In [13]:
#scale the dataset
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [14]:
#we'll use 80% of the data as training data 
train = int(len(dataset) * 0.80)
test = len(dataset) - train

In [15]:
print(train, test)

2155 539


In [16]:
train= dataset[:train]
test = dataset[len(train):]

In [17]:
train.shape

(2155, 1)

In [18]:
test.shape

(539, 1)

In [19]:
#I'll use past two days data to predict the price for next day. I tried a few numbers and this was giving least error. Therefore, I thought of using this.
x_train, y_train = [], []
for i in range(len(train)-2):
    x_train.append(dataset[i:i+2,0])
    y_train.append(dataset[i+2,0])
x_train, y_train = np.array(x_train), np.array(y_train)

In [20]:
x_train.shape

(2153, 2)

In [21]:
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
x_train.shape

(2153, 2, 1)

In [22]:
x_train.shape

(2153, 2, 1)

In [23]:
x_test = []
y_test=[]
for i in range(len(test)-2):
    x_test.append(dataset[len(train)-2+i:len(train)+i,0])
    y_test.append(dataset[len(train)+i,0])
x_test = np.array(x_test)
y_test = np.array(y_test)

In [24]:
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
x_test.shape

(537, 2, 1)

I will use LSTM to predict the stock price because LSTM stores the past information in predicting the future.

In [25]:
#I've used a LSTM model to predict the stock price. i checked for various other models but this was giving the least error. Therefore, I've used that
def create_model (learn_rate=0.005,neurons = 256) :
    model= Sequential([
                   LSTM(neurons, input_shape=(x_train.shape[1],1), return_sequences=True),
                   Dropout(0.4),
                   LSTM(neurons),
                   Dropout(0.2),
                   Dense(16, activation='relu'),
                   Dense(1)
    ])
    model.compile(loss='mean_squared_error', optimizer=Adam(lr = learn_rate) , metrics = ['mean_squared_error','accuracy'])
    return model
#print(create_model().summary())

In [26]:
batch_size = [10,20,40,50,80,100,128, 140,150,200,250,280,300]
epochs = [10,20,30,40,50,60,70,80,100,120,150,180,200,240,250,300,350,400]
learn_rate = [0.001, 0.005, 0.002, 0.003, 0.004, 0.006,0.007,0.008,0.01]
neurons = [1,10,15,20,25,50,75,100,125,150,175,200,225,256,270,280,300,350,400]
param_grid = dict(batch_size=batch_size, epochs=epochs,learn_rate=learn_rate, neurons=neurons)

In [27]:
from keras.wrappers.scikit_learn import KerasRegressor

In [28]:
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

In [29]:
from sklearn.model_selection import RandomizedSearchCV
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

E:\anaconda\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning



Best: nan using {'neurons': 125, 'learn_rate': 0.004, 'epochs': 100, 'batch_size': 300}
nan (nan) with: {'neurons': 125, 'learn_rate': 0.004, 'epochs': 100, 'batch_size': 300}
nan (nan) with: {'neurons': 20, 'learn_rate': 0.004, 'epochs': 10, 'batch_size': 80}
nan (nan) with: {'neurons': 175, 'learn_rate': 0.004, 'epochs': 250, 'batch_size': 40}
nan (nan) with: {'neurons': 175, 'learn_rate': 0.01, 'epochs': 70, 'batch_size': 10}
nan (nan) with: {'neurons': 25, 'learn_rate': 0.003, 'epochs': 120, 'batch_size': 10}
nan (nan) with: {'neurons': 20, 'learn_rate': 0.002, 'epochs': 150, 'batch_size': 200}
nan (nan) with: {'neurons': 256, 'learn_rate': 0.01, 'epochs': 100, 'batch_size': 150}
nan (nan) with: {'neurons': 50, 'learn_rate': 0.002, 'epochs': 10, 'batch_size': 20}
nan (nan) with: {'neurons': 15, 'learn_rate': 0.008, 'epochs': 300, 'batch_size': 280}
nan (nan) with: {'neurons': 225, 'learn_rate': 0.003, 'epochs': 240, 'batch_size': 300}


In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: ignored

In [ ]:
history = model.fit(x_train, y_train, epochs=100 , batch_size = 50, validation_data=(x_test, y_test))

In [ ]:
accuracy = model.evaluate(x_test,y_test)[1]
print(accuracy)

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

In [ ]:
#summarize history for error
fig, ax = plt.subplots(figsize=(9,4))
plt.plot(history.history['mean_squared_error'])
plt.plot(history.history['val_mean_squared_error'])
plt.title('model mean squared error')
plt.ylabel('Error')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
fig, ax = plt.subplots(figsize=(9,4))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
#using the model for x_test and then converting the data to normal price using inverse transform
predicted_price= model.predict(x_test)
pred = predicted_price[:, 0]
from sklearn.metrics import mean_squared_error, r2_score
print("R2 score : %.2f" % r2_score(y_test,pred))

In [ ]:
predicted_price = scaler.inverse_transform(predicted_price)
predicted_price

In [ ]:
len(predicted_price)

In [ ]:
predicted_price =np.array(predicted_price)
predicted_price.shape

In [ ]:
y_test.shape

In [ ]:
y_test

In [ ]:
#checking the score for our data
def model_score(model, X_train, y_train, X_test, y_test):
    trainScore = model.evaluate(X_train, y_train, verbose=0)
    print('Train Score: %.5f MSE (%.2f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))
    testScore = model.evaluate(X_test, y_test, verbose=0)
    print('Test Score: %.5f MSE (%.2f RMSE)' % (testScore[0], math.sqrt(testScore[0])))
    return trainScore[0], testScore[0]

model_score(model, x_train, y_train , x_test, y_test)

In [ ]:
predicted_price = predicted_price[:, 0]
predicted_price

Since the MSE and RMSE is quite less for both: the training as well as the test data, our model does a great job.

In [ ]:
predicted_price[:10]

In [ ]:
y_test = y_test.reshape(y_test.shape[0] , 1)
y_test = scaler.inverse_transform(y_test)
y_test[:10]

In [ ]:
#comparing the first 10 values of prediction for our data
diff = predicted_price-y_test
diff[:10]

In [ ]:
#plotting the courves for the actual test values and the predicted values. 
#The actual values are represented by the blue line and the predicted value by the red line
print("Red - Predicted Stock Prices  ,  Blue - Actual Stock Prices")
plt.rcParams["figure.figsize"] = (15,7)
plt.plot(y_test , 'b')
plt.plot(predicted_price , 'r')
plt.xlabel('Time')
plt.ylabel('Stock Prices')
plt.title('Check the accuracy of the model with time')
plt.legend(['Actual','Predict'], loc='upper right')
plt.grid(True)
plt.show()

The final plot gives the inference that our model has done pretty well in predicting the stock price for the chosen company(i.e. Microsoft).